# HW3

In [1]:
# Common imports
import numpy as np
import pandas as pd


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
home = pd.read_csv("home_data.csv")
home.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### 1. Polynomial Regression 
Split the original data into train-validate set (90%) and test set (10%) using random state = 0.  
Build a polynomial regression model with the degree = 15 using just ‘sqft_living’ on the train-validation set. 
Remember to run standard scaler to normalize the features before building your model.  

Report the model’s performance on the train-validate set and the test set.

In [3]:
from sklearn.model_selection import train_test_split
X = home.drop("price", axis=1)
y = home["price"]
X.shape

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state= 0)

(21613, 20)

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge, LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

pipe_poly = make_pipeline(PolynomialFeatures(degree=15), StandardScaler(), LinearRegression())
pipe_poly.fit(X_train_val[["sqft_living"]], y_train_val)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=15)),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [5]:
print("The score of the train-validate set: ", pipe_poly.score(X_train_val[["sqft_living"]], y_train_val))
print("The score of the test set:", pipe_poly.score(X_test[["sqft_living"]], y_test))

The score of the train-validate set:  0.5566378006085222
The score of the test set: 0.5150395945149853


### 2. Ridge Regression 
Split the original data into train-validate set (90%) and test set (10%) using random state = 0.  

For each L2_penalty λ in [10^3, 10^3.5, 10^4, 10^4.5, ..., 10^9], use GridSearchCV and 10-fold cross validation to compare the performance of the ridge regression with polynomial degree = 15 using just ‘sqft_living’ on the train-validation set.  Remember to run standard scaler to normalize  the features before building your model. 

Report which L2 penalty λ produced the lowest average validation error. Report the best model’s performance on the test set.

In [6]:

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state= 0)

In [7]:
from sklearn.model_selection import GridSearchCV

param_grid_ridge = {'ridge__alpha': np.logspace(3, 9, 13)}
print(param_grid_ridge)

{'ridge__alpha': array([1.00000000e+03, 3.16227766e+03, 1.00000000e+04, 3.16227766e+04,
       1.00000000e+05, 3.16227766e+05, 1.00000000e+06, 3.16227766e+06,
       1.00000000e+07, 3.16227766e+07, 1.00000000e+08, 3.16227766e+08,
       1.00000000e+09])}


In [8]:
pipe_poly_ridge = make_pipeline(PolynomialFeatures(degree=15),StandardScaler(),Ridge())
grid_ridge = GridSearchCV(pipe_poly_ridge, param_grid_ridge, cv=10, return_train_score= True)  # pip, grid, cv, return_train_score
grid_ridge.fit(X_train_val[["sqft_living"]], y_train_val)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures(degree=15)),
                                       ('standardscaler', StandardScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': array([1.00000000e+03, 3.16227766e+03, 1.00000000e+04, 3.16227766e+04,
       1.00000000e+05, 3.16227766e+05, 1.00000000e+06, 3.16227766e+06,
       1.00000000e+07, 3.16227766e+07, 1.00000000e+08, 3.16227766e+08,
       1.00000000e+09])},
             return_train_score=True)

In [9]:
pd.DataFrame(grid_ridge.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ridge__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,0.009702,0.002532,0.0017,0.000458,1000.0,{'ridge__alpha': 1000.0},0.485344,0.581764,0.667653,0.477012,...,0.529729,0.556348,0.557034,0.544872,0.548613,0.551367,0.553299,0.550867,0.549052,0.007685
1,0.008802,0.000600,0.0014,0.000490,3162.27766,{'ridge__alpha': 3162.2776601683795},0.485795,0.577205,0.664038,0.477918,...,0.527303,0.553900,0.554483,0.542046,0.546038,0.548927,0.550809,0.548395,0.546524,0.007701
2,0.009903,0.001578,0.0017,0.000458,10000.0,{'ridge__alpha': 10000.0},0.477658,0.554888,0.642261,0.470867,...,0.511663,0.537323,0.537971,0.527778,0.529498,0.532424,0.534393,0.531970,0.530339,0.007387
3,0.009202,0.000872,0.0017,0.000459,31622.776602,{'ridge__alpha': 31622.776601683792},0.417568,0.470496,0.556196,0.412845,...,0.440033,0.461482,0.462569,0.467482,0.454435,0.456973,0.459416,0.456811,0.456933,0.007240
4,0.008402,0.000663,0.0015,0.000500,100000.0,{'ridge__alpha': 100000.0},0.270953,0.301260,0.374251,0.269001,...,0.283574,0.299252,0.300534,0.329042,0.294418,0.295977,0.298518,0.295890,0.298636,0.011208
5,0.008102,0.000300,0.0014,0.000490,316227.766017,{'ridge__alpha': 316227.7660168379},0.125017,0.140498,0.186756,0.125245,...,0.132565,0.143263,0.144021,0.174681,0.140885,0.141568,0.143198,0.141298,0.144273,0.010632
6,0.008302,0.000640,0.0013,0.000458,1000000.0,{'ridge__alpha': 1000000.0},0.045472,0.051828,0.073606,0.046720,...,0.050259,0.055700,0.055997,0.072854,0.054773,0.055021,0.055739,0.054810,0.056425,0.005708
7,0.008002,0.000633,0.0015,0.000500,3162277.660168,{'ridge__alpha': 3162277.6601683795},0.013876,0.015903,0.024879,0.015486,...,0.017078,0.019181,0.019281,0.025869,0.018863,0.018946,0.019204,0.018853,0.019480,0.002218
8,0.007902,0.000700,0.0013,0.000458,10000000.0,{'ridge__alpha': 10000000.0},0.003005,0.003412,0.007467,0.004733,...,0.005538,0.006251,0.006283,0.008523,0.006148,0.006174,0.006260,0.006142,0.006354,0.000753
9,0.007902,0.000539,0.0012,0.000400,31622776.601684,{'ridge__alpha': 31622776.60168379},-0.000532,-0.000668,0.001724,0.001233,...,0.001766,0.001997,0.002007,0.002732,0.001963,0.001972,0.001999,0.001961,0.002030,0.000244


In [10]:
print(f"L2 penalty λ produced the lowest average validation error: ",grid_ridge.best_params_)
print(f"The score of the validate set on {grid_ridge.best_params_} is: ", grid_ridge.best_score_)
print(f"The score of the test set on {grid_ridge.best_params_} is: ", grid_ridge.score(X_test[["sqft_living"]],y_test))

L2 penalty λ produced the lowest average validation error:  {'ridge__alpha': 1000.0}
The score of the validate set on {'ridge__alpha': 1000.0} is:  0.5352353061240779
The score of the test set on {'ridge__alpha': 1000.0} is:  0.5108856832219166


### 3. Lasso Regression 
#### Part 1 
Create new features by performing following transformation on inputs: (assume you have named your data frame “sales”) 

from math import log, sqrt 

* sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt) 
* sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt) 
* sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms'] 
* sales['floors_square'] = sales['floors']*sales['floors'] 

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and 
lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly 
affect houses with many bedrooms. 

* On the other hand, taking square root of sqft_living will decrease the separation between big 
house and small house. The owner may not be exactly twice as happy for getting a house 
that is twice as big. 
 
Split the data into train-validate set (90%) and test set (10%) using random state = 0.  

Run Lasso regression with λ = 100 using the following features on the train-validate set. 
Remember to run standard scaler to normalize the features before building your model. 

['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 
'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',   
'sqft_basement', 'yr_built', 'yr_renovated'] 

Which features have been chosen by LASSO, i.e. which features were assigned nonzero 
weights? Report the model’s performance on the test set.  


In [11]:
from math import log, sqrt

home['sqft_living_sqrt'] = home['sqft_living'].apply(sqrt) 
home['sqft_lot_sqrt'] = home['sqft_lot'].apply(sqrt) 
home['bedrooms_square'] = home['bedrooms']*home['bedrooms'] 
home['floors_square'] = home['floors']*home['floors'] 
home.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_living_sqrt,sqft_lot_sqrt,bedrooms_square,floors_square
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,0,98178,47.5112,-122.257,1340,5650,34.351128,75.166482,9,1.0
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,1991,98125,47.7210,-122.319,1690,7639,50.695167,85.099941,9,4.0
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,0,98028,47.7379,-122.233,2720,8062,27.748874,100.000000,4,1.0
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,0,98136,47.5208,-122.393,1360,5000,44.271887,70.710678,16,1.0
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,0,98074,47.6168,-122.045,1800,7503,40.987803,89.888820,9,1.0


In [12]:
X = home.drop("price",axis=1)
y = home["price"]

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state= 0)

In [13]:
features = ['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 
'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',   
'sqft_basement', 'yr_built', 'yr_renovated'] 

In [14]:
X_train_val[features]

,bedrooms,bedrooms_square,bathrooms,sqft_living,sqft_living_sqrt,sqft_lot,sqft_lot_sqrt,floors,floors_square,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated
2763,3,9,2.50,1990,44.609416,4040,63.560994,1.5,2.25,0,0,5,8,1390,600,1926,0
16481,3,9,1.50,880,29.664794,6600,81.240384,1.0,1.00,0,0,5,6,880,0,1945,0
13726,3,9,2.25,1630,40.373258,7900,88.881944,1.0,1.00,0,0,4,7,1130,500,1973,0
8928,4,16,2.50,3360,57.965507,11548,107.461621,2.0,4.00,0,0,3,9,3360,0,1988,0
3494,3,9,3.00,2840,53.291650,206910,454.873609,2.0,4.00,0,0,3,10,2840,0,1999,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,5,25,5.00,3960,62.928531,94089,306.739303,2.0,4.00,0,0,3,10,3960,0,1998,0
19648,2,4,2.50,1400,37.416574,1262,35.524639,2.0,4.00,0,0,3,8,1160,240,2008,0
9845,3,9,2.25,2360,48.579831,14950,122.270193,1.0,1.00,0,0,4,9,2360,0,1978,0
10799,4,16,2.00,2370,48.682646,76665,276.884452,2.0,4.00,0,0,4,8,2370,0,1978,0


In [15]:
pipe_poly_lasso = make_pipeline(StandardScaler(),Lasso(alpha=100, max_iter=int(1e6)))
pipe_poly_lasso.fit(X_train_val[features], y_train_val)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('lasso', Lasso(alpha=100, max_iter=1000000))])

In [16]:
lasso = pipe_poly_lasso.named_steps["lasso"]
w = lasso.coef_
print("The weight of the model are: ", w)
print("The features whose the weight are not equal to 0 are : ", np.array(features)[list(np.where(w!=0)[0])] )

The weight of the model are:  [-1.68414025e+04  4.21990729e+03  3.94887063e+04  5.50537706e+05
 -4.10556428e+05  2.79001167e+04 -4.56092780e+04 -4.50525679e+04
  5.40846145e+04  5.09212322e+04  3.02941700e+04  1.78319600e+04
  1.52524907e+05  3.34446925e+01  0.00000000e+00 -9.51657456e+04
  7.32272887e+03]
The features whose the weight are not equal to 0 are :  ['bedrooms' 'bedrooms_square' 'bathrooms' 'sqft_living' 'sqft_living_sqrt'
 'sqft_lot' 'sqft_lot_sqrt' 'floors' 'floors_square' 'waterfront' 'view'
 'condition' 'grade' 'sqft_above' 'yr_built' 'yr_renovated']


In [17]:
print(f"The score of the test set: ",pipe_poly_lasso.score(X_test[features],y_test) )

The score of the test set:  0.6675759046117229


#### Part 2 (continue from Part 1) 

For each L1_penalty λ in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7], use GridSearchCV and 10-fold 
cross validation to compare the performance of the lasso regression using all the features used in 
Part 1 on the train-validation set. Remember to run standard scaler to normalize the features. 

Report which L1 penalty λ produced the lowest average validation error. Which features have 
been chosen by the best model, i.e. which features were assigned nonzero weights? Report the 
best model’s performance on the test set. 

In [18]:
param_grid_lasso = {"lasso__alpha": np.logspace(1,7,13)}
param_grid_lasso

{'lasso__alpha': array([1.00000000e+01, 3.16227766e+01, 1.00000000e+02, 3.16227766e+02,
        1.00000000e+03, 3.16227766e+03, 1.00000000e+04, 3.16227766e+04,
        1.00000000e+05, 3.16227766e+05, 1.00000000e+06, 3.16227766e+06,
        1.00000000e+07])}

In [22]:
pipe_lasso = make_pipeline(StandardScaler(),Lasso(max_iter=int(1e6)))
grid_lasso = GridSearchCV(pipe_lasso, param_grid_lasso, cv=10, return_train_score=True)
grid_lasso.fit(X_train_val[features], y_train_val)

c:\Users\anson\Documents\GitHub\SCU_ML\.env\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.815e+13, tolerance: 2.449e+11
  model = cd_fast.enet_coordinate_descent(


KeyboardInterrupt: 

In [20]:
pd.DataFrame(grid_lasso.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lasso__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,2.494637,0.011127,0.001700,0.000641,10.0,{'lasso__alpha': 10.0},0.678670,0.687395,0.730026,0.634527,...,0.679759,0.692209,0.692425,0.686497,0.686851,0.684996,0.687084,0.688090,0.687217,0.003387
1,2.567490,0.063696,0.001800,0.000400,31.622777,{'lasso__alpha': 31.622776601683793},0.678772,0.687234,0.729825,0.634560,...,0.679757,0.692207,0.692423,0.686495,0.686849,0.684993,0.687081,0.688088,0.687215,0.003387
2,2.500574,0.027715,0.001500,0.000500,100.0,{'lasso__alpha': 100.0},0.679078,0.686710,0.729172,0.634652,...,0.679734,0.692186,0.692403,0.686473,0.686828,0.684972,0.687046,0.688067,0.687192,0.003388
3,2.405252,0.029415,0.001400,0.000490,316.227766,{'lasso__alpha': 316.22776601683796},0.679900,0.684931,0.726953,0.634794,...,0.679508,0.691979,0.692202,0.686254,0.686614,0.684762,0.686743,0.687862,0.686971,0.003394
4,1.570960,0.117901,0.001701,0.000459,1000.0,{'lasso__alpha': 1000.0},0.681286,0.678821,0.718975,0.634874,...,0.677692,0.690458,0.690647,0.684521,0.684910,0.683211,0.685033,0.686244,0.685323,0.003462
5,0.464307,0.013647,0.001700,0.000458,3162.27766,{'lasso__alpha': 3162.2776601683795},0.677541,0.652537,0.683508,0.630221,...,0.663691,0.678024,0.678652,0.670785,0.671849,0.670266,0.672380,0.673609,0.672367,0.003974
6,0.052012,0.001733,0.001400,0.000490,10000.0,{'lasso__alpha': 10000.0},0.659519,0.617632,0.646208,0.614812,...,0.644132,0.650648,0.651644,0.643509,0.646493,0.643826,0.645831,0.647304,0.646839,0.002663
7,0.032807,0.001077,0.001300,0.000458,31622.776602,{'lasso__alpha': 31622.776601683792},0.629680,0.577243,0.600773,0.592229,...,0.608246,0.615386,0.617086,0.607352,0.610714,0.608160,0.609933,0.612491,0.611449,0.003076
8,0.027406,0.000800,0.001400,0.000490,100000.0,{'lasso__alpha': 100000.0},0.479916,0.419294,0.432964,0.449339,...,0.444754,0.454977,0.459260,0.450002,0.450328,0.449310,0.452343,0.454346,0.451858,0.003760
9,0.010703,0.000640,0.001400,0.000490,316227.766017,{'lasso__alpha': 316227.7660168379},-0.002185,-0.002577,-0.000974,-0.000402,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
lasso = grid_lasso.best_estimator_.named_steps["lasso"]
w = lasso.coef_
print("The weight of the model are: ", w)
print("The features whose the weight are not equal to 0 are : ", np.array(features)[list(np.where(w!=0)[0])] )

print(f"L1 penalty λ produced the lowest average validation error: ",grid_lasso.best_params_)
print(f"The score of the test set: ", grid_lasso.score(X_test[features],y_test) )

The weight of the model are:  [ -16776.57035489    3740.43007989   38747.4186551   379217.89299771
 -395121.29855452   25565.78429072  -42954.31241735  -29034.99124526
   39016.35571312   50819.32380009   30564.35017491   17542.26794053
  152153.62466036  139441.57935266   75576.6686674   -94948.22236464
    7058.37362861]
The features whose the weight are not equal to 0 are :  ['bedrooms' 'bedrooms_square' 'bathrooms' 'sqft_living' 'sqft_living_sqrt'
 'sqft_lot' 'sqft_lot_sqrt' 'floors' 'floors_square' 'waterfront' 'view'
 'condition' 'grade' 'sqft_above' 'sqft_basement' 'yr_built'
 'yr_renovated']
L1 penalty λ produced the lowest average validation error:  {'lasso__alpha': 316.22776601683796}
The score of the test set:  0.6672079738695584
